In [1]:
import pandas as pd
import json
from preprocessing import preprocess
from functools import reduce


In [2]:
df_train = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_test = pd.read_csv('datasets/test_radiomics_hipocamp.csv')

In [3]:
df_train = preprocess(df_train,mode="all")

In [8]:
x,y = df_train
x_columns = enumerate(list(x.columns))
x_columns = list(map(lambda tup:tup[1]+str(tup[0]),x_columns))
x_columns.sort()
x_columns = list(map(lambda x:x.split("_"),x_columns))
x_columns

[['Age2012'],
 ['Sex2011'],
 ['diagnostics', 'Image-original', 'Maximum1'],
 ['diagnostics', 'Image-original', 'Mean0'],
 ['diagnostics', 'Mask-original', 'VoxelNum2'],
 ['exponential', 'firstorder', '10Percentile1598'],
 ['exponential', 'firstorder', '90Percentile1599'],
 ['exponential', 'firstorder', 'Energy1600'],
 ['exponential', 'firstorder', 'Entropy1601'],
 ['exponential', 'firstorder', 'InterquartileRange1602'],
 ['exponential', 'firstorder', 'Kurtosis1603'],
 ['exponential', 'firstorder', 'Maximum1604'],
 ['exponential', 'firstorder', 'Mean1606'],
 ['exponential', 'firstorder', 'MeanAbsoluteDeviation1605'],
 ['exponential', 'firstorder', 'Median1607'],
 ['exponential', 'firstorder', 'Minimum1608'],
 ['exponential', 'firstorder', 'Range1609'],
 ['exponential', 'firstorder', 'RobustMeanAbsoluteDeviation1610'],
 ['exponential', 'firstorder', 'RootMeanSquared1611'],
 ['exponential', 'firstorder', 'Skewness1612'],
 ['exponential', 'firstorder', 'TotalEnergy1613'],
 ['exponential', 

In [5]:

def recursive_merge(dict1, dict2):
    """
    Recursively merge two dictionaries. Combines sub-dictionaries where both have the same key.
    """
    result = dict1.copy()  # Start with a shallow copy of the first dictionary
    for key, value in dict2.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = recursive_merge(result[key], value)
        else:
            result[key] = value
    return result

def gen_trie(l):
    if l == []:
        return dict()
    else:
        head = l[0]
        tail = l[1:]

        if head == []:
            return gen_trie(tail)
        else:
            h_head = head[0]
            h_tail = head[1:]
            rec = gen_trie(tail)
            return recursive_merge({h_head:gen_trie([h_tail])} ,rec)

cols_trie = gen_trie(x_columns)
with open("cols_trie.json","+w") as f:
    json.dump(cols_trie,f,indent=2,sort_keys=True)
def tree_size(tree):
    res = 0
    for v in tree.values():
        res += 1
        if v != {}:
            res += tree_size(v)
    return res

def n_leaves(tree):
    res = 0
    for v in tree.values():
        if v != {}:
            res += tree_size(v)
        else:
            res += 1
    return res

In [31]:
list(cols_trie.keys())
res = []
for k,v in cols_trie.items():
    res = res + [(k,n_leaves(v))]
res

[('Age', 0),
 ('Sex', 0),
 ('diagnostics', 3),
 ('exponential', 93),
 ('gradient', 93),
 ('lbp-2D', 44),
 ('lbp-3D-k', 93),
 ('lbp-3D-m1', 44),
 ('lbp-3D-m2', 46),
 ('log-sigma-1-0-mm-3D', 93),
 ('log-sigma-2-0-mm-3D', 93),
 ('log-sigma-3-0-mm-3D', 93),
 ('log-sigma-4-0-mm-3D', 93),
 ('log-sigma-5-0-mm-3D', 93),
 ('logarithm', 93),
 ('original', 107),
 ('square', 93),
 ('squareroot', 93),
 ('wavelet-HHH', 93),
 ('wavelet-HHL', 93),
 ('wavelet-HLH', 93),
 ('wavelet-HLL', 93),
 ('wavelet-LHH', 93),
 ('wavelet-LHL', 93),
 ('wavelet-LLH', 93),
 ('wavelet-LLL', 93)]